## ① Google Driveにアップロードしたテキスト（学習用データ）にアクセスできるようにする

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/'My Drive'
%ls

## ②globライブラリ
要インストール:ノート内で%pip install globとうって実行してください。

glo.glob関数を用いて、学習用に用いるテキストファイルをlist形式で得る。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import glob
files = glob.glob("./akutagawa/corpus/*.txt")
print(len(files))

197


## ③Mecabのインストールと形態素解析
以下のコマンド3行よりインストール。

Mecabを用いて文章を単語ごとに分割します、がその前に元の文章には不要な記号や記述、学習を妨げる特殊な文字（いわゆる外れ値）があるので、
まずreライブラリを用いて消去してます。

その後、Mecab.Tagger()ドライバを用いてフォルダにあるテキストファイル20個までについて形態素解析（単語へ分割）したのち、一つのリスト(text_all)へ格納。できればファイルの数だけ読み込みをしたかったがデータ量が莫大になってしまったため20個に断念。

In [ ]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

In [ ]:
import re
def making_data(file):
  with open(file,mode='r',encoding='shift_jis', errors='ignore') as f:
    text_original = f.read()

  text = re.sub("《[^》]+》", "", text_original) # ルビの削除
  text = re.sub("［[^］]+］", "", text) # 読みの注意の削除
  text = re.sub(".*-", "", text, flags=re.DOTALL) #-より上の部分の削除
  text = re.sub("（[大正|明治.年]+.*", "", text, flags=re.DOTALL) #＃地付きより下の部分の削除
  text = re.sub("\n.\n", "", text, flags=re.DOTALL) #題名の削除
  text = re.sub("\n", "", text) #改行の削除
  text = re.sub("[0-9]+|[０-９]+", "", text, flags=re.DOTALL) #数字の削除
  text = re.sub("[Ａ-Ｚ]+|[A-Z]+|[a-z]+", "", text, flags=re.DOTALL) #全角英語の削除
  text = re.sub("[｜ 　―\,\，./「」\：\:（）＊×…※△『』〜＼／]", "", text) # 各種記号の削除
  text = re.sub("\[\［", "", text)
  text = re.sub("\]\］", "", text)





  return text

In [ ]:
import MeCab
m = MeCab.Tagger('-Owakati')
text_all = []
for i in range(20):
    text = making_data(files[i])
    text = m.parse(text)
    text = text.split(' ')
    text_all.extend(text)

## ④使用されている単語の辞書の作成

得られた単語リストを集合に変換したのちソートをかけることで、辞書ができます。この辞書（chars）を用いて後にone-hot表現を実現します。

char_indicesはキーが文字、バリューがcharsのその文字のあるインデックス、indices_charはキーがインデックス、バリューがcharsのそのインデックスに対応した文字の辞書を定義しておきます。

In [ ]:
chars = sorted(list(set(text_all)))
char_indices = {}
indices_char = {}

for i, char in enumerate(chars):
    char_indices[char] = i
    indices_char[i] = char

In [ ]:
print(chars)

['\n', '、', '。', '々', 'あ', 'ああ', 'あい', 'あいつ', 'あがく', 'あがり', 'あきらめ', 'あきらめる', 'あくまで', 'あくまでも', 'あけ', 'あけよ', 'あける', 'あげ', 'あさり', 'あし', 'あすこ', 'あそこ', 'あたかも', 'あたし', 'あたっ', 'あたり', 'あだな', 'あっ', 'あっち', 'あっと言う間に', 'あつ', 'あつめ', 'あと', 'あなた', 'あに', 'あの', 'あばら家', 'あぶなく', 'あまり', 'あや', 'あやまっ', 'あやまり', 'あやまん', 'あら', 'あらう', 'あらし', 'あらゆる', 'あらわ', 'あらわし', 'あり', 'ありとあらゆる', 'ある', 'あるい', 'あるいは', 'あれ', 'あろ', 'あわす', 'あんな', 'あんなに', 'あんまり', 'い', 'いい', 'いいえ', 'いう', 'いえ', 'いかなる', 'いかに', 'いかにも', 'いがみ合い', 'いきなり', 'いくつ', 'いくばく', 'いくら', 'いけ', 'いささか', 'いざ', 'いじめ', 'いずれ', 'いそいそ', 'いた事', 'いだき', 'いっ', 'いっしょ', 'いっぱい', 'いつ', 'いつか', 'いつのまにか', 'いつの間にか', 'いつも', 'いなせ', 'いなそ', 'いふ', 'いまさら', 'いや', 'いやいや', 'いよいよ', 'いら', 'いらだたしい', 'いらっしゃい', 'いらっしゃら', 'いらっしゃる', 'いらっしゃれ', 'いる', 'いるか', 'いれ', 'いろ', 'いろいろ', 'いわ', 'いわゆる', 'いわんや', 'いん', 'う', 'うか', 'うかい', 'うけ', 'うさ', 'うし', 'うす', 'うすい', 'うすら寒く', 'うそ寒', 'うたい', 'うたう', 'うたっ', 'うだつ', 'うち', 'うっ', 'うっかり', 'うつ', 'うつむい', 'うとうと', 'うに', 'うねり', 'うまい', 'うらうら', 'うらがなしい', 'うるさく', 'うれしかっ', 'うろうろ', 'うろつ

## ⑤ニューロン数、バッチ数などの定義と学習用時系列データと正解データの作成
time_chars : 学習用データ : 時系列の数(n_rnn)だけ連続でテキストを入手、次の行には要素（単語）一つずらしたテキストが入る。これをもとの文章の長さ分だけやりたいが時系列の長さ分最後にはみ出てしまうので時系列の数分引いた数だけのループとなる。

next_chars : 正解データ : 学習用データの各行に対して次に並んでいる単語を格納。

これにより
学習用データには
\begin{bmatrix}
[text\_all[0] … text\_all[9]]\\
[text\_all[1] … text\_all[10]\\
[text\_all[len(text\_all)-n\_rnn] … text\_all[len(text\_all)-1]
\end{bmatrix}

正解データには
\begin{bmatrix}
[text\_all[10] \\
[text\_all[11]\\
[text\_all[len(text\_all)]]
\end{bmatrix}

In [ ]:
n_rnn = 10
n_in = 1
n_mid = 256
n_out = 1
epochs = 30
batch_size = 8

In [ ]:
time_chars = []
next_chars = []
for i in range(0, len(text_all)-n_rnn):
    time_chars.append(text_all[i: i+n_rnn])
    next_chars.append(text_all[i + n_rnn])

In [ ]:
len(time_chars)

62741

## ⑥データをOne-hot表現へ

正解用データ、学習用データそれぞれに単語辞書の長さ分の次元を追加する。
その入れ子をx、tとする。

一つ目のfor文にて学習用データの行の数だけループを回す。ここでchar_indicesを用いて正解データの辞書でのインデックスを取得し、tの該当するインデックスに1をセットする。これで各行の単語に該当するインデックスだけ1の配列が完成。

２つ目のfor分では学習用データの各行(i)に対して時系列の数だけループを回す。ここでも正解データと同様な操作を行い、時系列データそれぞれの単語に対応したインデックスが1となる配列の完成。


In [ ]:
x = np.zeros((len(time_chars), n_rnn, len(chars)), dtype=np.bool)
t = np.zeros((len(time_chars), len(chars)), dtype=np.bool)

for i,t_cs in enumerate(time_chars):
    t[i, char_indices[next_chars[i]]] = 1
    for j, char in enumerate(t_cs):
        x[i, j, char_indices[char]] = 1

## ⑦RNNモデルの構築
RNNの構造についてSimpleRNN,LSTM,GRUの選択肢があったが計算量が少ないかつ精度が比較的良いGRUを用いることにした。
活性化関数や損失関数等はOne-hot表現に適したものを選んだ。

In [ ]:
from keras.layers import GRU, Dense
from keras.models import Sequential

model_gru = Sequential()
model_gru.add(GRU(n_mid, input_shape=(n_rnn, len(chars))))
model_gru.add(Dense(len(chars), activation="softmax"))
model_gru.compile(loss='categorical_crossentropy', optimizer='adam')
print(model_gru.summary())


## ⑧各エポックごとに学習度合いを確認する関数を作成

モデル学習時にLambdaCallbackインスタンスを渡すことでエポック前後での挙動を追加できる。今回はepoch終了時にon_epoch_end関数を実行することになってる。

またEarlystoppingインスタンスを渡すことにより学習が進まなくなった時に早期終了させることができる。今回はval_loss（検証用誤差）が変化しなくなってから5回epoch学習を行ったのち終了させる。

In [ ]:
from keras.callbacks import LambdaCallback, EarlyStopping
import copy

def on_epoch_end(epoch, logs):
    print('エポック：',epoch)

    beta = 5
    index = np.random.choice(len(time_chars))
    prev_text = text_all[index:n_rnn+index]
    created_text = copy.copy(prev_text)

    model.save('akutagawa.h5')

    print('シード：', created_text)

    for i in range(200):
        x_pred = np.zeros((1, n_rnn, len(chars)))

        for j, char in enumerate(prev_text):
            x_pred[0, j, char_indices[char]] = 1
        
        y = model.predict(x_pred)
        p_power = y[0] ** beta
        next_index = np.random.choice(len(chars), p=p_power/np.sum(p_power))
        next_char = indices_char[next_index]

        created_text += next_char
        prev_text = prev_text[1:] + [next_char]
    
    print(created_text)
    print()

epoch_end_callback = LambdaCallback(on_epoch_end = on_epoch_end)
early_stopping = EarlyStopping(monitor="val_loss", patience=5)


## ⑨モデルのロードと学習の実行
初めての学習であればmodel変数にはmodel_gruを入れます。
保存済みモデルを使う場合はコメント部分を使います。

In [ ]:
# from keras.models import load_model
# model = load_model('akutagawa.h5')
model = model_gru
history_gru = model.fit(x, t, 
                        batch_size = batch_size,
                        epochs = epochs,
                        callbacks = [epoch_end_callback, early_stopping])

## ⑩モデルの保存
モデルの保存を行います。ただ、この操作はLambdaCallbackの関数にて指定しており毎epochごとに保存がなされますのでこの行で実行しなくてもいいです。

In [ ]:
model.save('akutagawa.h5')